# Quantum Brilliance Qristal Challenge
The goal of this challenge is to solve a puzzle that is actually a small optimization problem in disguise, using a well-known quantum algorithm.  That algorithm is the so-called exponential search, also known as Grover's algorithm. This Jupyter notebook guides you through the steps needed to create your own version of this algorithm. We will explore what a qubit is, how to manipulate qubits, and which subroutines are needed for the Grover algorithm. 
 
If you get stuck, you can find solutions in the same folder as this file.

## Getting Started

### Qubits and Quantum Circuits

Measuring the state of a qubit ends its role in the calculation. This is in contrast to classical bits, whose values can be read and used elsewhere whilst allowing the calculation to continue.
A qubit is generally in a superposition of $|0\rangle$ and $|1\rangle$, which are the computational basis states.  However, only one of these two values is returned upon measurement. The probability to measure one or the other is given by the absolute square of the respective amplitude $$\vert\psi\rangle = \cos\frac{\theta}{2}\vert 0\rangle + e^{i \phi} \sin\frac{\theta}{2}\vert1\rangle$$ $$0<\theta<\pi,\ 0<\phi<2\pi$$
<center><img src="Bloch3.png" width="400" height="400">

The instructions to manipulate qubits are often referred to as a "quantum circuit". The qubits are usually initialized in the $|0\rangle$ state. The circuit typically consists of a sequence of operations called "gates" acting on one or more qubits. Therefore, this model of quantum computing is also called "gate-based" quantum computing. In the final layer of the quantum circuit one specifies which qubits are to be measured.

As the measurement outcomes of superposition states are random, one often needs to run the circuit multiple times. These repetitions are called shots. From the distribution of results that one obtains over a series of shots, one can then compute expectation values or extract only the most likely outcome. The precision of the expectation value or the confidence in a certain sample depends on the number of shots i.e, the shot noise.

Qubits are fragile objects. Any interaction with the environment can destroy the quantum properties of the qubit very much like a measurement. The longer the qubit can maintain its quantumness -- its coherence -- the more operations can be applied to it. The Quantum Brilliance emulator can include this noise in its simulations if desired.

### Circuits: Basic Methods

| Name | Inputs | Outputs | Descriptions |
| --- | --- | --- | --- |
| print | None | None  | Print the quantum circuit that has been built. |
| openqasm | None | str | Get the OpenQASM representation of the circuit. |
| append | arg0: qbos.Circuit | None | Append another quantum circuit to this circuit. |
| h | qubit: int | None | Add Hadamard gate |
| x | qubit: int | None | Add Pauli-X gate |
| y | qubit: int | None | Add Pauli-Y gate |
| z | qubit: int | None | Add Pauli-Z gate |
| u1 | qubit: int, theta: float | None | Add $U_1(\theta)$ gate |
| u3 | qubit: int, theta: float, phi: float, lambda:float | None | Add $U_3(\theta, \phi, \lambda)$ gate |
| t | qubit: int | None | Add T gate |
| tdg | qubit: int | None | Add T-dagger gate |
| s | qubit: int | None | Add S gate |
| sdg | qubit: int | None | Add S-dagger gate |
| rx | qubit: int, theta: float | None | Add rotation around X gate ($R_x(\theta)$) |
| ry | qubit: int, theta: float | None | Add rotation around Y gate ($R_y(\theta)$) |
| rz | qubit: int, theta: float | None | Add rotation around Z gate ($R_y(\theta)$) |
| cx | control_qubit: int, target_qubit: int | None | Add Controlled-X gate |
| mcx | control_qubits: array[int], target_qubit: int | None | Add Multi-controlled-X gate (control qubits are given as a list of indices) |
| generalised_mcx | target: int, controls_on: array[int], controls_off: array[int] | None | Add Multi-controlled-X gate conditional on controls_on being $\vert1\rangle$ and controls_off being $\vert0\rangle$. |
| amcu | U: CircuitBuilder, control_qubits: array[int], ancilla_qubits: array[int] | None | Add multi-controlled-U gate using ancillary qubits |
| ccx | control_qubit1: int, control_qubit2: int, target_qubit: int | None | Add CCNOT (Toffoli gate) |
| swap | qubit1: int, qubit2: int | None | Add SWAP gate |
| controlled_swap | qubits_a: array[int], qubits_b: array[int], flags_on: array[int], flags_off: array[int] | None | Add SWAP operation between register a and register b conditional on on flags being $\vert1\rangle$ and off flags being $\vert0\rangle$. |
| cphase | control_qubit: int, target_qubit: int, theta: float | None | Add controlled-phase rotation gate (also known as $CU_1(\theta)$ gate) |
| cz | control_qubit: int, target_qubit: int | None | Add Controlled-Z gate |
| ch | control_qubit: int, target_qubit: int | None | Add Controlled-Hadamard gate |
| measure | qubit: int | None | Add Measure to a qubit |
| measure_all | None | None | Add Measure to all qubits in the circuit |

### Creating circuits using QASM
There are several options to create quantum circuits. A very low-level language is the Quantum Assembly Language (QASM) https://en.wikipedia.org/wiki/OpenQASM.

#### Example 1: Initialise and measure a qubit
In the QASM string one specifies the names of the qubit and classical registers. By default, qubits are initialised to $\vert0\rangle$, so there is nothing to be done. At the end of every quantum circuit one needs to specify which qubits are to be measured. We then create a QB SDK session and specify a few parameters, such as the number of shots, the number of qubits, and the backend (hardware or simulted hardware) on which the circuit should be executed. We also must assign the circuit, and finally run it.

In [ ]:
#Import the Qristal core
import qristal.core

# Define the quantum program to run (aka 'quantum kernel' aka 'quantum circuit')
initialize_and_measure = '''
__qpu__ void MY_QUANTUM_CIRCUIT(qreg q)
{
  OPENQASM 2.0;
  include "qelib1.inc";
  creg c[1];
  measure q[0] -> c[0];
}
'''

# Create a quantum computing session using the QB SDK
my_sim = qristal.core.session()

# Set the number of qubits
my_sim.qn = 1

# Choose a simulator backend
my_sim.acc = "aer"

# Choose how many 'shots' to run through the circuit
my_sim.sn = 1000

# Choose to output results for all possible bit combinations, not just those measured
my_sim.calc_all_bitstring_counts = True

# assign the circuit to instring
my_sim.instring = initialize_and_measure

# Run the circuit 1000 times and count up the results in each of the classical registers
print("About to run quantum program...")
my_sim.run()
print("Ran successfully!")

# Print the cumulative results in each of the classical registers
print("Results:")
print(my_sim.results)

#### Example 2: Include Noise
Next, we will run the same circuit again, but this time we include noise. The QB SDK Emulator can emulate different sources of noise. The sources for errors in quantum computers are initialisation, measurement and calibration errors, as well as decoherence.

In [ ]:
# Turn on the noise
my_sim.noise = True

# Run the circuit 1000 times and count up the results in each of the classical registers
print("About to run quantum program...")
my_sim.run()
print("Ran successfully!")

# Print the cumulative results in each of the classical registers
print("Results:")
print(my_sim.results)

# Turn off the noise for now
my_sim.noise = False

Real quantum computers are physical devices. Here we can see that real qubits are not always perfectly initialised or measured.

In [ ]:
# Plot a histogram of the result
import numpy as np
import matplotlib.pyplot as plt
from pylab import figure, show, legend, ylabel
fig1 = figure()
ax1 = fig1.add_subplot(111)
xdat=[format(iix,'01b') for iix in range(len(my_sim.all_bitstring_counts))]
ydat=my_sim.all_bitstring_counts
ax1.bar(xdat,ydat)
plt.yticks(np.arange(0, 1001, step=100))
plt.xlabel("States")
plt.ylabel("Shots measured")

#### Exercise 1: Gate Operation in QASM
Explore the action of the x gate. What would you call the analogous classical gate?
Hint: Use the table at the beginning of this section.

In [ ]:
### Create a quantum circuit with an gate and measure using QASM code. Make sure you name your circuit "x_gate_circuit".
####

x_gate_circuit = '''
__qpu__ void MY_QUANTUM_CIRCUIT(qreg q)
{
  OPENQASM 2.0;
  include "qelib1.inc";
  creg c[1];
  x q[0];
  measure q[0] -> c[0];
}
'''

####

#### Exercise 2: The Hadamard gate with QASM
The Hadamard gate plays a special role among the single qubit gates. It generates an equal superposition state, which can be understood as generating a change of basis. After plotting your results you should see approximately 50% 0s and 50% 1s. https://en.wikipedia.org/wiki/Quantum_logic_gate#Hadamard_gate

In [ ]:
### Create a quantum circuit with a Hadamard gate and measure. Run the circuit 500 times and plot the results.
####

h_circuit = '''
__qpu__ void MY_QUANTUM_CIRCUIT(qreg q)
{
  OPENQASM 2.0;
  include "qelib1.inc";
  creg c[1];
  h q[0];
  measure q[0] -> c[0];
}
'''

my_sim.instring = h_circuit
my_sim.sn = 1000
# Run the circuit 1000 times and count up the results in each of the classical registers
print("About to run quantum program...")
my_sim.run()
print("Ran successfully!")

# Print the cumulative results in each of the classical registers
print("Results:")
print(my_sim.results)

# Plot a histogram of the result
import numpy as np
import matplotlib.pyplot as plt
from pylab import figure, show, legend, ylabel
fig1 = figure()
ax1 = fig1.add_subplot(111)
xdat=[format(iix,'01b') for iix in range(len(my_sim.all_bitstring_counts))]
ydat=my_sim.all_bitstring_counts
ax1.bar(xdat,ydat)
plt.yticks(np.arange(0, 1001, step=100))
plt.xlabel("States")
plt.ylabel("Shots measured")
####

#### Exercise 3: Quantum vs. classical coin flips
Given the result above, one is tempted interpret the action of the Hadamard gate as a quantum coin flip. However, as can be seen in this exercise, two consecutive Hadamard operations yield a very different result compared to the classical analogue of flipping a coin twice. Interpret the result.

In [ ]:
### Create a quantum circuit with two Hadamard gates. Measure.
### Run 500 shots, print the results and try to find out what happened.
####

two_h = '''
__qpu__ void MY_QUANTUM_CIRCUIT(qreg q)
{
  OPENQASM 2.0;
  include "qelib1.inc";
  creg c[1];
  h q[0];
  h q[0];
  measure q[0] -> c[0];
}
'''

my_sim.instring = two_h
my_sim.sn = 500

# Run the circuit
print("About to run quantum program...")
my_sim.run()
print("Ran successfully!")

# Print the cumulative results
print("Results:")
print(my_sim.results)

#### Exercise 4: The entangling gate
The true power of quantum computing comes from combining the two fundamental principles: superposition and entanglement. The simplest entangling gates act on two qubits.  For many quantum computing platforms, this is also the maximal number of qubits a so-called native entangling gate can act on. On many of the platforms, controlled NOT gates are standard, and we will also use them here. https://en.wikipedia.org/wiki/Quantum_logic_gate#Controlled_gates

Now we will create the $\vert \Phi^+\rangle$-Bell state! https://en.wikipedia.org/wiki/Bell_state

In [ ]:
### Create a quantum circuit that generates the Bell state |Phi+>. Run the circuit 500 times and plot the results.
####

phi_plus = '''
__qpu__ void MY_QUANTUM_CIRCUIT(qreg q)
{
  OPENQASM 2.0;
  include "qelib1.inc";
  creg c[2];
  h q[0];
  cx q[0], q[1];
  measure q[0] -> c[0];
  measure q[1] -> c[1];
}
'''

my_sim.instring = phi_plus
my_sim.sn = 500
my_sim.qn = 2
# Run the circuit 500 times and count up the results in each of the classical registers
print("About to run quantum program...")
my_sim.run()
print("Ran successfully!")

# Print the cumulative results in each of the classical registers
print("Results:")
print(my_sim.results)

# Plot a histogram of the result
import numpy as np
import matplotlib.pyplot as plt
from pylab import figure, show, legend, ylabel
fig1 = figure()
ax1 = fig1.add_subplot(111)
xdat=[format(iix,'01b') for iix in range(len(my_sim.all_bitstring_counts))]
ydat=my_sim.all_bitstring_counts
ax1.bar(xdat,ydat)
plt.yticks(np.arange(0, 1001, step=100))
plt.xlabel("States")
plt.ylabel("Shots measured")

#### Example 4: Initialise and measure a qubit using the Quantum Brilliance Circuit Builder
The QB SDK provides a "Circuit" class for users to construct quantum circuits from elementary gates, such as X, Y, Z, Hadamard, CNOT, etc. One simply appends one gate after the other. Before running the quantum circuit one assigns it using the "ir_target" to the session. Make sure that the "instring" method is cleared.

In [ ]:
# Create an empty circuit with a x gate
simple_circuit = qristal.core.Circuit()
# Add an x gate on qubit 0
simple_circuit.x(0)
# Measure all qubits in the circuit
simple_circuit.measure_all()

# Choose a simulator backend
my_sim.acc = "qpp"
# Choose how many 'shots' to run through the circuit
my_sim.sn = 100
# Choose how many qubits to simulate
my_sim.qn = 1
# Set the circuit, taking care to remove the instring from the previous example
my_sim.instring = ""
my_sim.irtarget = simple_circuit
# Run the circuit
my_sim.run()
print("Results:")
print(my_sim.results)

#### Exercise 5: More Bell states
Create all of the 4 Bell states $\vert \Phi^+\rangle$, $\vert \Phi^-\rangle$, $\vert \Psi^+\rangle$, $\vert \Psi^-\rangle$ using the circuit builder.
Hint: start recreating your circuit in Exercise 3 and apply the appropriate single qubit gates at the right places. Note that in the results, the relative phase in the different Bell states cannot be observed.

In [ ]:
### Create quantum circuits that generate the Bell state |Phi+> with the QB circuit builder. Run the circuit and print the results.

####

phi_plus = qristal.core.Circuit()
phi_plus.h(0)
phi_plus.cnot(0,1)
phi_plus.measure_all()

phi_minus = qristal.core.Circuit()
phi_minus.h(0)
phi_minus.cnot(0,1)
phi_minus.z(0)
phi_minus.measure_all()

psi_plus = qristal.core.Circuit()
psi_plus.h(0)
psi_plus.cnot(0,1)
psi_plus.x(0)
psi_plus.measure_all()

psi_minus = qristal.core.Circuit()
psi_minus.h(0)
psi_minus.cnot(0,1)
psi_minus.z(0)
psi_minus.x(0)
psi_minus.measure_all()

for circuit in [phi_plus, phi_minus, psi_plus, psi_minus]:
  my_sim = qristal.core.session()
  my_sim.sn = 500
  my_sim.qn = 2
  my_sim.irtarget = circuit
  my_sim.run()
  print("Results:")
  print(my_sim.results,"\n")

### Get access to a high-dimensional space
We will now start to explore how quantum computing can help with combinatorial optimization problems. The Hadamard gate creates an equal weighted superposition of the $\vert0\rangle$ and $\vert1\rangle$ states. Hence one qubit gives access to a (complex) two dimensional vector space (restricted to vectors of unit length, due to the need for all probabilities to sum to one). Let us see how we can access a $2^n$-dimensional space using only $n$ qubits.
Consider a multidimensional decision problem. Each of the $2^n$ possible decisions can be represented in a quantum bitstring of length $n$.

#### Exercise 6: Create an equal superposition of $n$ qubits.
Take care to create a void function i.e., one where the circuit is defined and manipulated only, and no return value is given.

In [ ]:
### Create a quantum circuit with an equal superposition state on n qubits.
# Note: do not change the function name!

def state_preparation(circuit, n_qubits):
    ####

    # Add Hadamard gate on each qubit
    for i in range(n_qubits):
        circuit.h(i)
        
    ####

### Run the circuit 1000 times and print, for n_qubits = 4.

# Choose a simulator backend
my_sim.acc = "qpp"
# Choose how many 'shots' to run through the circuit
my_sim.sn = 1000
# Choose how many qubits to simulate
my_sim.qn = 4
#set the circuit
state_prep = qristal.core.Circuit()
state_preparation(state_prep, my_sim.qn)
state_prep.measure_all()
my_sim.irtarget = state_prep
#run the circuit
my_sim.run()
print("Results:")
print(my_sim.results)

## Amplify the desired state
It is nice that qubits allow us to represent all possible bit stings at once, but what we really want is to find the most optimal or desired state. As a first step, we need to mark this state somehow (or mark several desired states). We can achieve this by multiplying those states with a negative sign. Operations that distinguish certain states are often referred to as oracles. https://cnot.io/quantum_algorithms/grover/grovers_algorithm.html

### Oracles
Note that the z gate only changes the sign of the $\vert1\rangle$ state. Using controlled z-gates, which also become active if the control qubit is in the $\vert1\rangle$ state, we can construct oracles. Consider for example the equal superposition state of 2 qubits.
$$cz:\  \vert\psi\rangle = \frac{1}{2}\left(\vert00\rangle + \vert01\rangle + \vert10\rangle + \vert11\rangle\right)\rightarrow\frac{1}{2}\left(\vert00\rangle + \vert01\rangle + \vert10\rangle - \vert11\rangle\right)$$
The cz-gate marks the $\vert11\rangle$ state only.

We can also mark any other state by operating with x gates before and after the action of the cz gate on the appropriate qubits.
    
Next let us consider 3 qubits. The available gate set only offers multi-controlled x gates. However, we can easily construct a multi-controlled z gate using the Hadamard gate. Here we exploit the relation: $z = h x h$

#### Example 4: Oracle for the 010 state

In [ ]:
### Create a quantum circuit that marks the 010 state by inverting the sign of its coefficient. It should take any previously prepared circuit as an input argument.

def oracle(circuit):
    circuit.x(0)
    circuit.x(2)
    circuit.h(2)
    circuit.mcx([0,1],2)
    circuit.h(2)
    circuit.x(0)
    circuit.x(2)

### Amplification (Diffusion)
Now that we have marked the desired state, we need to amplify its quantum amplitude to give it a high probability to appear when the system is measured. We will again do this using a reflection, this time about the mean probability of the state. How this is constructed as a gate sequence is bit more complicated, and details can be found in the reference at the beginning of the section. The construction principle is always the same: apply Hadamard gates to all query qubits, flip all qubits, flip the sign if all qubits are in the one state, flip all qubits, and then apply Hadamard gates to all query qubits.

#### Example 5: Diffusion Circuit

In [ ]:
def diffusion_circuit(circuit, n_qubits):
    for i in range(n_qubits):
        circuit.h(i)
    for i in range(n_qubits):
        circuit.x(i)
    circuit.h(n_qubits-1)
    circuit.mcx(list(range(n_qubits-1)),n_qubits-1)
    circuit.h(n_qubits-1)
    for i in range(n_qubits):
        circuit.x(i)    
    for i in range(n_qubits):
        circuit.h(i)

#### Example 6: The Grover Algorithm
- Run the state preparation: consider all possible solutions
- Mark the desired state with an oracle: apply a controlled operation (possibly checking the state of an additional qubit register)
- Amplify that state.
  
For $n$ qubits, the oracle and diffusion steps are repeated $\pi\sqrt{2^n}/4$ times in order to maximize the amplification. The best known classical algorithms scale linearly, i.e. the Grover search algorithm gives a quadratic speedup for searches in unstructured databases. In our example with 3 qubits, we only need one iteration.

In [ ]:
my_sim.qn = 3

# Grover algorithm
my_circuit = qristal.core.Circuit()
state_preparation(my_circuit, my_sim.qn)
oracle(my_circuit)
diffusion_circuit(my_circuit, my_sim.qn)
my_circuit.measure_all()

# Choose a simulator backend
my_sim.acc = "qpp"
# Choose how many 'shots' to run through the circuit
my_sim.sn = 1000
# Set the circuit
my_sim.irtarget = my_circuit
# Turn off optimisation as the mcx gates in the oracle and diffusion circuits cannot be processed by the circuit optimiser.
my_sim.nooptimise = True
# Run the circuit
my_sim.run()
print("Results:")
print(my_sim.results)

The state $\vert010\rangle$ is the most likely one.

## QB Challenge
### Switch off the lights

A similar challenge to this was given at the Qiskit Quantum Challenge in fall 2020. It beautifully shows how to construct more interesting oracles and how quantum computing can yield an advantage over classical algorithms.

The task given to you is to switch off all the lights on a given game board with 9 buttons in a 3x3 array.
<center><img src="light board.png" width="400" height="400">

The difficulty is that pushing a button not only switches the state of that tile, but also that of all adjacent tiles. For example, pushing button #6 also switches tiles #3 and #7.  The figure below demonstrates this.
    
<center><img src="push button 6.png" width="400" height="400">
    
In this challenge we will use the superposition principle to create all possible combinations of pushes, and use entanglement to construct an oracle that finds that combination that switches off all lights.

The starting light pattern corresponding to the image above is given by:

In [ ]:
lights = [0,0,0,1,0,1,1,1,0]

For your convenience we also specify a dictionary that contains which adjacent tiles are switched if a certain button is pushed.

In [ ]:
target_tiles_dict = {0:[0,1,3], 1: [0,1,2,4], 2: [1,2,5], 3: [0,3,4,6], 4: [1,3,4,5,7], 5: [2,4,5,8], 6:[3,6,7], 7:[4,6,7,8], 8: [5,7,8]}

#### Exercise 7: prepare ancillary qubits (light board)
The following function will read in the light board. We will use additional so-called "ancillary" qubits to construct the oracle that marks the solution that switches off the lights. 

In [ ]:
def prepare_ancillary_state(circuit, lights, n_pushes, n_lights):
    ####

    for i in range(n_pushes, n_pushes + n_lights):
        if lights[i-n_pushes] == 1:
            circuit.x(i)
    
    ####        

#### Exercise 8: construct the first part of the oracle
The following function switches tiles according to the rules of the game.

In [ ]:
def push_buttons(circuit, n_pushes, n_lights, target_tiles_dict):
    ####

    for i in range(n_pushes):
        for adjacent_tile in target_tiles_dict[i]:
            circuit.cnot(i, n_pushes + adjacent_tile)
    
    ####        

#### Exercise 9: Mark the state
Now we create an oracle that marks the state with a negative sign if all lights are off. Make sure that you undo any manipulations except for the multicontrolled z-gate, because we need to run the oracle several times. The lights register must return to its original state so that we can run the query again in each iteration. This step is called "uncompute".

In [ ]:
def oracle_light_out(circuit, n_pushes, n_lights):
    ####

    for i in range(n_pushes, n_pushes + n_lights):
        circuit.x(i)
    circuit.h(n_pushes + n_lights - 1)
    circuit.mcx(list(range(n_pushes, n_pushes + n_lights-1)), n_pushes + n_lights-1)
    circuit.h(n_pushes + n_lights - 1)
    for i in range(n_pushes, n_pushes + n_lights):
        circuit.x(i)

    ####    

#### Exercise 10: Grover Iterations
Specify the optimal number of Grover iterations.
Hint: The optimal push sequence is '000011001'. Can you find it?

In [ ]:
#import the Qristal core
import qristal.core

# Specify the optimal number of Grover iterations. Observe how the number of iterations can influence the result.
####

# Enter your code here!

n_iterations = 

####

my_circuit = qristal.core.Circuit()

# Prepare the push qubits. We consider all possible combinations of pushes in superposition
state_preparation(my_circuit, 9)
# Prepare the light board to be switched off
prepare_ancillary_state(my_circuit, lights, 9, 9)

# Gover iterations: oracle followed by the diffusion setp. Note, that we have to uncompute the push button step!
for i in range(n_iterations):
    push_buttons(my_circuit, 9, 9, target_tiles_dict)
    oracle_light_out(my_circuit, 9, 9)
    push_buttons(my_circuit, 9, 9, target_tiles_dict)
    diffusion_circuit(my_circuit, 9)   
for i in range(9):
    my_circuit.measure(i)

# Create a quantum computing session using the Qristal SDK
my_sim = qristal.core.session()
# Set the number of qubits
my_sim.qn = 18
my_sim.noplacement = True
my_sim.nooptimise = True
my_sim.output_oqm_enabled = False
# We will use the aer simulator
my_sim.acc = "aer"
# Choose how many 'shots' to run through the circuit
my_sim.sn = 1000
#set the circuit
my_sim.irtarget = my_circuit
#run the circuit
my_sim.run()
maxval = 0
maxstring = []
for bitstring in my_sim.results:
    if my_sim.results[bitstring] > maxval:
        maxval = my_sim.results[bitstring]
        maxstring = bitstring
print('The bit string with the most counts is: ', maxstring)
print("Results:")
print(my_sim.results)


## Thank You!
Thank you for trying out the Quantum Brilliance Qristal Challenge. Now that you have some hands-on experience, we invite you to take it to the next level by exploring the Qristal SDK, Qristal Emulator and Qristal virtual QPU (vQPU) for yourself.